<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/183-Mini-Project-M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai python-dotenv langchain langchain_openai langchain_community langchain_chroma langchain_text_splitters

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document  # Import the Document class

# Sample texts
texts = [
    "광대버섯(Amanita phalloides)은 크고 인상적인 후성(위) 자실체(담자과체)를 가지고 있습니다.",
    "큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.",
    "A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.",
]

# Custom text loader function
def load_texts(text_list):
    documents = []
    for text in text_list:
        documents.append(Document(page_content=text, metadata={}))  # Create Document objects
    return documents

# Load the documents
raw_documents = load_texts(texts)

# Initialize the text splitter and embeddings
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
embeddings = OpenAIEmbeddings()

# Split the documents into chunks
chunks = []
for doc in raw_documents:
    chunks.extend(text_splitter.split_text(doc.page_content))

# Embed each chunk
embedded_chunks = embeddings.embed_documents([chunk for chunk in chunks])  # Pass list of strings

# Initialize the vector store with embedded chunks
vector_store = Chroma.from_documents(
    [Document(page_content=chunk, metadata={}) for chunk in chunks],
    embeddings
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
def dynamic_search(question):
    try:
        k = int(input("원하는 검색 결과의 수를 입력하세요 (1-3): "))
        if k < 1 or k > 3:
            raise ValueError("검색 결과의 수는 1에서 3 사이여야 합니다.")
    except ValueError as e:
        print(f"잘못된 입력입니다: {e}")
        return []

    # Ensure vector_store is properly initialized and accessible
    return vector_store.similarity_search_with_score(query=question, k=k)

# 사용자 질문에 따른 검색
question = "큰 자실체를 가진 순백색 버섯에 대해 알려주세요"
search_results = dynamic_search(question)
print("검색 결과:", search_results)


원하는 검색 결과의 수를 입력하세요 (1-3): 3
검색 결과: [(Document(page_content='큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.'), 0.20683683454990387), (Document(page_content='A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.'), 0.3165876865386963), (Document(page_content='광대버섯(Amanita phalloides)은 크고 인상적인 후성(위) 자실체(담자과체)를 가지고 있습니다.'), 0.31780287623405457)]
